In [6]:
import os
import pickle
import pandas as pd
import numpy as np
import xgboost as xgb
from calculate_confusion_matrix import calculate_confusion_matrix
import time
from copy import deepcopy
from utils import one_hot_encoder

save_results = True
do_stratify = False

n_boot = 100
split = 0.7
np.random.seed(seed=0)

ft_dir = 'features_long/'

# list feature files
# files = os.listdir(ft_dir)
# files.remove('1515656.dat')
# print('subject 1515656 removed due to spoofed gps.')

# reading top locations
with open('top_locations.dat', 'rb') as f:
    location_top = pickle.load(f)
f.close()

feature_all = []
target_all = []

for filename in files:
    with open(ft_dir+filename, 'rb') as f:  
        feature, target = pickle.load(f)

        # only keeping top locations
        ind = np.array([], int)
        for (i,loc) in enumerate(target['location']):
            if loc in location_top:
                ind = np.append(ind, i)
        feature = feature.loc[ind,:]
        target = target.loc[ind]
        feature = feature.reset_index(drop=True)
        target = target.reset_index(drop=True)
        
        feature_all.append(feature)
        target_all.append(target)
        
    f.close()

confs = []
aucs = []
labels = []
inds = np.arange(0,len(feature_all),1)
inds_split = int(np.floor(split*len(feature_all)))

for i in range(n_boot):
    
    print('------------------')
    print(i)
#     if i==6:
#         print 'subject skipped because of lack of data'
#         continue
    
    # training set
    np.random.shuffle(inds)
    ind_train = inds[:inds_split]
    ind_test = inds[inds_split:]
    
    x_train = pd.concat([feature_all[j] for j in ind_train], axis=0)
    y_train = pd.concat([target_all[j]['location'] for j in ind_train], axis=0)
    x_train = x_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    
    # test set
    x_test = pd.concat([feature_all[j] for j in ind_test], axis=0)
    y_test = pd.concat([target_all[j]['location'] for j in ind_test], axis=0)
    x_test = x_test.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)
    
    # remove foursquare features
    x_train = x_train.drop(['fsq 0','fsq 1','fsq 2','fsq 3','fsq 4','fsq 5','fsq 6','fsq 7','fsq 8','fsq distance'],axis=1)
    x_test = x_test.drop(['fsq 0','fsq 1','fsq 2','fsq 3','fsq 4','fsq 5','fsq 6','fsq 7','fsq 8','fsq distance'],axis=1)
    x_train = x_train.reset_index(drop=True)
    x_test = x_test.reset_index(drop=True)
    
    # model (sensor)
    gbm = xgb.XGBClassifier(max_depth=6, n_estimators=50, learning_rate=0.05, nthread=12, subsample=0.25, \
                        colsample_bytree=0.5, max_delta_step=0, gamma=3, objective='mlogloss', reg_alpha=0.5, \
                        missing=np.nan)
    # model (sensor + foursquare)
#     gbm = xgb.XGBClassifier(max_depth=6, n_estimators=75, learning_rate=0.05, nthread=12, subsample=0.25, \
#                         colsample_bytree=0.2, max_delta_step=0, gamma=3, objective='mlogloss', reg_alpha=0.5, \
#                         missing=np.nan)
    
    # fitting model
#     gbm.fit(x_train, y_train, eval_set=[(x_train,y_train),(x_test, y_test)], eval_metric='mlogloss', verbose=True)
#     print gbm.evals_result()
    gbm.fit(x_train, y_train)
    
    # training performance
    y_pred = gbm.predict(x_train)
    conf_train, roc_auc_train = calculate_confusion_matrix(y_pred, y_train)

    # test
    y_pred = gbm.predict(x_test)
    conf, roc_auc = calculate_confusion_matrix(y_pred, y_test)
    
    labels.append(np.unique(y_test))
    confs.append(conf)
    aucs.append(roc_auc)

    print(np.unique(y_test))
    print(roc_auc_train, np.nanmean(roc_auc_train))
    print(roc_auc, np.nanmean(roc_auc))

# saving the results
if save_results:
    with open('auc_location_new_10fold3_1515656removed_sensor.dat','wb') as f:
        pickle.dump([aucs, confs, labels], f)
    f.close()


------------------
0


/usr/local/lib/python3.4/dist-packages/sklearn/metrics/ranking.py:542: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


["Another's Home" 'Arts & Entertainment (Theater, Music Venue, Etc.)'
 'Food (Restaurant, Cafe)' 'Home' 'Nightlife Spot (Bar, Club)'
 'Outdoors & Recreation' 'Spiritual (Church, Temple, Etc.)' 'Work']
[ 0.88304628  0.94187414  0.75819259  0.94641293  0.96119535  0.97423148
  0.99425932  0.88590719] 0.91813990881
[ 0.78844896  0.493663    0.72910518  0.92109013  0.99273407  0.48983192
         nan  0.80317874] 0.745435998303
------------------
1


/usr/local/lib/python3.4/dist-packages/sklearn/metrics/ranking.py:542: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


["Another's Home" 'Arts & Entertainment (Theater, Music Venue, Etc.)'
 'Food (Restaurant, Cafe)' 'Home' 'Nightlife Spot (Bar, Club)'
 'Outdoors & Recreation' 'Spiritual (Church, Temple, Etc.)' 'Work']
[ 0.84171881  0.96527997  0.76409134  0.94444433  0.93708913  0.98953467
  0.99443047  0.88532113] 0.915238732593
[ 0.64521551  0.56486964  0.71415239  0.88488441  0.87024624         nan
         nan  0.80860626] 0.747995741376
------------------
2


...

/usr/local/lib/python3.4/dist-packages/sklearn/metrics/ranking.py:542: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


["Another's Home" 'Arts & Entertainment (Theater, Music Venue, Etc.)'
 'Food (Restaurant, Cafe)' 'Home' 'Nightlife Spot (Bar, Club)'
 'Outdoors & Recreation' 'Spiritual (Church, Temple, Etc.)' 'Work']
[ 0.85488125  0.96751355  0.7638748   0.95817721  0.95384322  0.97438157
  0.99504993  0.89892401] 0.920830692193
[ 0.65956664  0.49130708  0.72360143  0.88991915  0.74601761  0.65881162
         nan  0.77996602] 0.707027079506
------------------
98


/usr/local/lib/python3.4/dist-packages/sklearn/metrics/ranking.py:542: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


["Another's Home" 'Arts & Entertainment (Theater, Music Venue, Etc.)'
 'Food (Restaurant, Cafe)' 'Home' 'Nightlife Spot (Bar, Club)'
 'Outdoors & Recreation' 'Spiritual (Church, Temple, Etc.)' 'Work']
[ 0.83085402  0.94267432  0.76203661  0.94390045  0.92514837  0.93013868
  0.99387071  0.89304197] 0.902708141936
[ 0.61910392  0.65855589  0.7207205   0.89460968  0.49614396         nan
  0.9958457   0.8016696 ] 0.740949892384
------------------
99
["Another's Home" 'Arts & Entertainment (Theater, Music Venue, Etc.)'
 'Food (Restaurant, Cafe)' 'Home' 'Nightlife Spot (Bar, Club)'
 'Outdoors & Recreation' 'Spiritual (Church, Temple, Etc.)' 'Work']
[ 0.85696594  0.96129511  0.76374576  0.94342301  0.99493035  0.97314074
         nan  0.88016791] 0.910524116059
[ 0.69890445  0.70693403  0.72307963  0.90010494  0.99291204  0.49188312
         nan  0.77922089] 0.756148442708


/usr/local/lib/python3.4/dist-packages/sklearn/metrics/ranking.py:542: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


In [2]:
ft_dir+filename

'features_long/NQ003SV.dat'

In [ ]:
a = ['b','b','a','b','b','b','a']
b = ['a','a','a','b','b','b','c']
calculate_confusion_matrix(a,b)